In [ ]:
# library imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
import tqdm

%matplotlib inline

## Import Data

In [ ]:
# set the path of the raw data
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
train_file_path = os.path.join(raw_data_path, 'train_house.csv')
test_file_path = os.path.join(raw_data_path, 'test_house.csv')

In [ ]:
# read the data with all default parameters
train_df = pd.read_csv(train_file_path, index_col='Id')
test_df = pd.read_csv(test_file_path, index_col='Id')

## Organizing data

In [ ]:
# get columns with nan values from training data
# numerical columns and categorical columns in the training data
col_with_nan = [col for col in train_df.columns if train_df[col].isnull().any()]
#num_col = [col for col in train_df.columns if train_df[col].dtypes in ['float64', 'int64']]
#cat_col = list(set(train_df.columns) - set(num_col))

## Basic Structure

In [ ]:
train_df.info()

In [ ]:
test_df.info()

## Check for duplicates

In [ ]:
# Try to find duplicated data
duplicatedID = train_df.duplicated()
train_df[duplicatedID] #no duplicated found

## Study the data graphically

In [ ]:
def hist_log_hist (df, col, show=True, file_path=''):
    
    '''
        Returns the histogram and log1p of the values histogram of the column col
        with their probability plots.
        
        Args:
            df: Dataframe with features. Shape(m, n)
            col: feature to be plotted
            show: display graphics if True

    '''

    f, axes = plt.subplots(2, 2, figsize=(12, 10));

    # original histogram
    sns.distplot(df[df[col].notnull()][col], fit=norm, ax=axes[0, 0]);
    stats.probplot(df[df[col].notnull()][col], plot=axes[1, 0]);

    # log-histogram (deals with zero and NaN values)
    sns.distplot(np.log1p(df[df[col].notnull()][col]), fit=norm, ax=axes[0, 1]);
    stats.probplot(np.log1p(df[df[col].notnull()][col]), plot=axes[1, 1]);
    
    plt.suptitle(col, fontsize=18)
    plt.tight_layout(rect=[0, 0, 1, 0.95]);
    
    if show:
        plt.show();
    elif file_path:
        plt.savefig(file_path, dpi=300)

In [ ]:
# distributions of numerical features
# saving histograms for all numerical columns to see skewness of data

for col in tqdm.tqdm(num_col):
    
    path = os.path.join(os.path.pardir, 'data', 'figures', col)
    path = '.'.join([path, 'png'])
    hist_log_hist(train_df, col, show=False, file_path=path)
    plt.cla()
    plt.clf()
    plt.close()

In [ ]:
# very skewed columns without 'SalePrice'
# skew_col = ['1stFlrSF', 'GrLivArea', 'LotArea']

In [ ]:
# check scatter plot with the target "SalePrice"
for col in tqdm.tqdm(num_col):
    
    path = os.path.join(os.path.pardir, 'data', 'figures', col)
    path = '.'.join([path, 'png'])
    plt.scatter(train_df[col], train_df['SalePrice'])
    plt.savefig(path, dpi=300)
    plt.cla()
    plt.clf()
    plt.close()

Removing outiliers shown in the scatter plot. Points that did not follow the patterns in the scatter plot were dropped

In [ ]:
# outliers from scatter plot
# point > 4000 in 1stFlrSF
# point > 5000 in BsmtFinSF1
# point >~ 4500 in GrLivArea
# point > 100000 in LotArea
# point > 300 in LotFrontage
# drop outliers from above

outliers_removed = train_df.copy()
outliers_removed = outliers_removed.drop(outliers_removed[outliers_removed['1stFlrSF'] >= 4000].index)
outliers_removed = outliers_removed.drop(outliers_removed[outliers_removed.BsmtFinSF1 >= 5000].index)
outliers_removed = outliers_removed.drop(outliers_removed[outliers_removed.GrLivArea > 4500].index)
outliers_removed = outliers_removed.drop(outliers_removed[outliers_removed.LotArea > 100000].index)
outliers_removed = outliers_removed.drop(outliers_removed[outliers_removed.LotFrontage > 300].index)

Check how each categorical feature influence the target sale price for the houses

In [ ]:
# check importance of categorical features
for col in tqdm.tqdm(cat_col):
    
    path = os.path.join(os.path.pardir, 'data', 'figures', col)
    path = '.'.join([path, 'png'])
    
    # order bars by its mean
    order = outliers_removed.groupby([col])['SalePrice'].aggregate(
        np.mean).reset_index().sort_values('SalePrice')
    
    plt.figure(figsize=(12, 7))
    
    sns.barplot(x=col, y='SalePrice', data=outliers_removed, order=order[col]);
    
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.cla()
    plt.clf()
    plt.close()

Keeping track of the freatures that showed some ranking in the categories

In [ ]:
# # columns with clear rankings
# rank_col = ['Alley', 'BsmtCond', 'BsmtExposure', 'BsmtQual',
#             'CentralAir', 'Electrical', 'ExterCond', 'Condition2', 
#             'Exterior1st', 'Exterior2nd', 'ExterQual', 'Fence', 
#             'FireplaceQU', 'Foundation', 'Functional', 'GarageFinish', 
#             'GarageQual', 'GarageType', 'Heating', 'HeatingQC', 'HouseStyle',
#             'KitchenQual', 'LandContour', 'MasVnrType', 'MSZoning', 'Neighborhood', 
#             'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleType', 'Street',
#             'Utilities']
# # maybe target encoder columns (not so clear ranking) or one hot encoder
# te_col = ['BldgType', 'BsmtFinType1', 'BsmtFinType2', 'Condition1', 'GarageCond', 
#           'LandSlope', 'LotConfig', 'LotShape', 'MiscFeature', 'SaleCondition']

Checking numerical features that are actually categorical

In [ ]:
num_cat_col = ['MSSubClass', 'OverallQual', 'OverallCond']

In [ ]:
for col in tqdm.tqdm(num_cat_col):
    
    path = os.path.join(os.path.pardir, 'data', 'figures', col)
    path = '.'.join([path, 'png'])
    
    # order bars by its mean
    order = outliers_removed.groupby([col])['SalePrice'].aggregate(
        np.mean).reset_index().sort_values('SalePrice')
    
    plt.figure(figsize=(12, 7))
    
    sns.barplot(x=col, y='SalePrice', data=outliers_removed, order=order[col]);
    
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.cla()
    plt.clf()
    plt.close()

From the figures we can see that MSSubClass is not actually in order, so we will pass it to categorical feature.

In [ ]:
# MSSubClass
train_df['MSSubClass'] = train_df['MSSubClass'].astype('object')
test_df['MSSubClass'] = test_df['MSSubClass'].astype('object')

# Year and Month sold are transformed into categorical features.
train_df['YrSold'] = train_df['YrSold'].astype('object')
test_df['YrSold'] = test_df['YrSold'].astype('object')
train_df['MoSold'] = train_df['MoSold'].astype('object')
test_df['MoSold'] = test_df['MoSold'].astype('object')

## Missing values

In [ ]:
train_df = outliers_removed.copy()
train_df[col_with_nan].info()

In [ ]:
# missing values dataframe to avoid errors in code
missing = train_df.copy()

It is probable that houses have similar LotFrontage in the same Neighborhood

In [ ]:
# fill missing values of lotfrontage with values of the median in the neighborhood
missing['LotFrontage'] = missing.groupby(['Neighborhood'])['LotFrontage'].transform(
    lambda x: x.fillna(x.median()))

All these features are probably not in the houses with missing values, so we will impute 'NA'.

In [ ]:
# filling NaN values with 'NA'
fill_NA_col = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2',
              'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'Alley',
              'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

for col in fill_NA_col:
    missing[col].fillna('NA', inplace=True)

For MasVnrArea and Type and Electrical we will impute the most frequent values because there are few missing values. For the the GarageYrBlt we will impute zero, because there is no garage in the house.

In [ ]:
# fill with the most frequent value
mode_MasVnr = missing[['MasVnrArea','MasVnrType']].mode().iloc[0,:]
missing.MasVnrArea.fillna(mode_MasVnr[0], inplace=True)
missing.MasVnrType.fillna(mode_MasVnr[1], inplace=True)

# fill the one mv with the most frequent value in the column
mode_Electrical = missing.Electrical.mode()[0]
missing.Electrical.fillna(mode_Electrical, inplace=True)

# filling with year with zero because the garage was not built
missing['GarageYrBlt'].fillna(0, inplace=True)

In [ ]:
train_df = missing.copy()

## Missing Values for test data

In [ ]:
# get cols with NaN values
test_col_with_nan = [col for col in test_df.columns if test_df[col].isnull().any()]

In [ ]:
# protecting original data
test_missing = test_df.copy()

In [ ]:
test_missing[test_col_with_nan].info()

Get columns with categorical features and numerical ones. We will use it to fill missing values in the test data separately.

In [ ]:
# getting numerical and categorical cols in cols with nan
cat_col_with_nan = [col for col in test_col_with_nan if test_missing[col].dtypes == 'object']
num_col_with_nan = list(set(test_col_with_nan) - set(cat_col_with_nan))

In [ ]:
# filling LotFrontage missing values in test data with training median values
for neighborhood in list(train_df.groupby(['Neighborhood'])['LotFrontage'].median().index):
    
    median = train_df[train_df['Neighborhood'] == neighborhood].median()['LotFrontage']
    
    test_missing.loc[test_missing['Neighborhood'] == neighborhood, 'LotFrontage'] = \
        test_missing.loc[test_missing['Neighborhood'] == neighborhood, 'LotFrontage'].fillna(median)

In [ ]:
# replacing missing value in the test data with the most frequent data in train data
# for categorical features, and with the median for numerical features
for col in cat_col_with_nan:
    test_missing[col].fillna(train_df[col].mode()[0], inplace=True)

for col in num_col_with_nan:
    test_missing[col].fillna(train_df[col].median(), inplace=True)

In [ ]:
# separating 'object', 'int64' and 'float64' columns
cat_col = [col for col in train_df.columns if train_df[col].dtypes == 'object']
float_col = [col for col in train_df.columns if train_df[col].dtypes == 'float64']
int_col = [col for col in train_df.columns if train_df[col].dtypes == 'int64']

In [ ]:
# fixing dtypes of columns in test data
for col in test_missing.columns:
    
    if col in cat_col:
        test_missing = test_missing.astype({col: 'object'})
    elif col in int_col:
        test_missing = test_missing.astype({col: 'int64'})
    elif col in float_col:
        test_missing = test_missing.astype({col: 'float64'})

In [ ]:
test_df = test_missing.copy()

## Preparing data

Concatenate train and test to manipulate features

In [ ]:
y = train_df['SalePrice']
train_df = train_df.drop('SalePrice', axis=1)
all_data = pd.concat([train_df, test_df])

There is only one sample in the training set that has Utilities as 'NoSeWa'. So it will not use this feature.

In [ ]:
all_data.groupby('Utilities')['Utilities'].count()

In [ ]:
all_data = all_data.drop('Utilities', axis=1)

Adding some features that may improve the models predictions

In [ ]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
all_data['TotalPorchArea'] = (all_data['EnclosedPorch'] + all_data['OpenPorchSF'] + 
                              all_data['3SsnPorch'] + all_data['ScreenPorch'])
all_data['HasPool'] = all_data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
all_data['Has2ndFloor'] = all_data['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
all_data['HasGarage'] = all_data['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
all_data['HasBsmt'] = all_data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
all_data['YearBuilt'] = all_data['YearBuilt'].astype('object')
all_data['YearRemodAdd'] = all_data['YearRemodAdd'].astype('object')
all_data['GarageYrBlt'] = all_data['GarageYrBlt'].astype('object')

In [ ]:
# Label Encoding ranked features
NA_Ex_dict = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
label_encoding = {'Alley': {'NA': 0, 'Grvl': 1, 'Pave': 2}, 
                 'ExterQual': NA_Ex_dict,
                 'ExterCond': NA_Ex_dict,
                 'BsmtQual': NA_Ex_dict,
                 'BsmtCond': NA_Ex_dict,
                 'HeatingQC': NA_Ex_dict,
                 'KitchenQual': NA_Ex_dict,
                 'FireplaceQu': NA_Ex_dict,
                 'GarageQual': NA_Ex_dict,
                 'GarageCond': NA_Ex_dict,
                 'PoolQC': NA_Ex_dict,
                 'BsmtExposure':{'No': 0, 'Gd': 3, 'Mn': 1, 'Av': 2, 'NA': 0},
                 'CentralAir': {'N': 0, 'Y': 1},
                 'GarageFinish': {'RFn': 2, 'Unf': 1, 'Fin': 3, 'NA': 0},
                 'PavedDrive': {'Y': 2, 'N': 0, 'P': 1},
                 'Fence': {'NA': 0, 'MnPrv': 3, 'GdWo': 2, 'GdPrv': 4, 'MnWw': 1},
                 'Electrical': {'SBrkr': 5, 'FuseF': 3, 'FuseA': 4, 'FuseP': 2, 'Mix': 1}}

all_data.replace(label_encoding, inplace=True)

In [ ]:
# get remaining object cols
object_col = [col for col in all_data.columns if all_data[col].dtypes == 'object']
num_col = [col for col in all_data.columns if all_data[col].dtypes != 'object']

In [ ]:
skewed_feats = all_data[num_col].apply(lambda x: skew(x)).sort_values(ascending=False)

skewed_feats

Deal with skewness in the data

In [ ]:
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

def fixing_skewness(df):
    """
    This function takes in a dataframe and return fixed skewed dataframe
    """
    
    ## Getting all the data that are not of "object" type. 
    numeric_feats = df.dtypes[df.dtypes != "object"].index

    # Check the skew of all numerical features
    skewed_feats = df[numeric_feats].apply(lambda x: skew(x)).sort_values(ascending=False)
    high_skew = skewed_feats[abs(skewed_feats) > 0.5]
    skewed_features = high_skew.index

    for feat in skewed_features:
        df[feat] = boxcox1p(df[feat], boxcox_normmax(df[feat] + 1))

fixing_skewness(all_data)

In [ ]:
y = np.log1p(y)

In [ ]:
train_df = all_data[:len(train_df)]
test_df = all_data[len(train_df):]

X = train_df.copy()
X_test = test_df.copy()

In [ ]:
# Target Encoding the remaining object columns
import category_encoders as ce

target_encoder = ce.TargetEncoder(cols = object_col)

target_encoder.fit(X[object_col], y)
X[object_col] = target_encoder.transform(X[object_col], y)
X_test[object_col] = target_encoder.transform(X_test[object_col])

In [ ]:
# Normalization

from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()
X = robust_scaler.fit_transform(X)
X_test = robust_scaler.transform(X_test)

In [ ]:
X = pd.DataFrame(X)
X_test = pd.DataFrame(X_test)

## Training Model

### Ridge Model

In [ ]:
# Ridge model
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha= 0.01, normalize=True)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = np.sqrt(-1*cross_val_score(ridge_model, X, y, cv=5, scoring='neg_mean_squared_error'))
np.mean(scores)

### ElasticNet Model

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net_model = ElasticNet(alpha= 0.0001, normalize=True)

scores2 = np.sqrt(-1*cross_val_score(elastic_net_model, X, y, cv=5, scoring='neg_mean_squared_error'))
np.mean(scores2)

### Lasso Model

In [ ]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=0.0001, normalize=True)

scores3 = np.sqrt(-1*cross_val_score(lasso_model, X, y, cv=5, scoring='neg_mean_squared_error'))
np.mean(scores3)

### LightGBM model with SelectKBest

With X, y and y_test in hands, now we can make a model and train the data to make predictions.

First, let's split the data into training and validation data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  test_size=0.2, 
                                                  random_state=0)

In [ ]:
# checking distribution of splitting
print(y_train.mean())
print(y_val.mean())

In [ ]:
# imports
from sklearn.feature_selection import SelectKBest, mutual_info_regression
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

Define functions to select the best parameters and to evaluate score with mean absolute error

In [ ]:
def select_k_best(X_train, y_train, X_val, k):
    # select only 10 best features
    selector = SelectKBest(mutual_info_regression, k=k)

    # array w/o columns
    X_k_train = selector.fit_transform(X_train, y_train)

    # get dataframe with unused columns with zeros
    selected_features = pd.DataFrame(selector.inverse_transform(X_k_train), 
                                     index=X_train.index, 
                                     columns = X_train.columns)

    selected_columns = selected_features.columns[selected_features.var() != 0]

    X_k_train = X_train[selected_columns]
    X_k_val = X_val[selected_columns]

    return (X_k_train, X_k_val)

# LightGBM model
def lgb_model(X_train, y_train, X_val, y_val):
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_val, label=y_val)

    param = {'num_leaves':64,
             'objective':'regression_l1', 
             'metric':'regression_l1', 
             'seed':7
            }

    bst = lgb.train(param, dtrain,
                    num_boost_round = 1000, 
                    valid_sets=[dvalid],
                    early_stopping_rounds=10,
                    verbose_eval=False)
    return(bst)

# calculates score of model
def score_rmse(X, y, bst):
    pred = bst.predict(X)
    score = np.sqrt(mean_squared_error(y, pred))
    return(score)

Find how many parameters give best results

In [ ]:
dict_score = {}

for k in tqdm.tqdm([10, 20, 30, 40, 50, 60, 70, 84]):
    
    X_k_train, X_k_val = select_k_best(X_train, y_train, X_val, k);
    
    model = lgb_model(X_k_train, y_train, X_k_val, y_val);
    
    dict_score[k] = score_rmse(X_k_val, y_val, model);

In [ ]:
dict_score

From the models used ElasticNet model showed the best result

In [ ]:
elastic_net_model.fit(X, y)
pred = elastic_net_model.predict(X_test)
pred = np.expm1(pred)

In [ ]:
# create output to submit to Kaggle
output = pd.DataFrame({'SalePrice': pred}, index=test_df.index)
output.to_csv('/home/artur/titanic/data/external/elastic_net_model.csv')